###Deltalake & Lakehouse Optimization Usecases

![](/Workspace/Users/infoblisstech@gmail.com/databricks-code-repo/5_all_databricks_workouts/DELTA OPTIMIZATIONS.png)

####1. Handling Data Skew & Query Performance (Optimize & Z-Order)
Scenario: The analytics team reports that queries filtering silver_shipments by source_city and shipment_date are becoming slow as data volume grows.

Task: Run the OPTIMIZE command with ZORDER on the silver_shipments table to co-locate related data in the same files.

Outcome:
Why did we choose source_city and shipment_date for Z-Ordering instead of shipment_id? Think about high cardinality vs. query filtering

In [0]:
%sql
use prodcatalog_wd36.logistics_wd36

In [0]:
%sql
optimize silver_shipments zorder by (destination_city);
    

In [0]:
%sql
describe history silver_shipments

In [0]:
%sql
select * from silver_shipments limit 2;

In [0]:
%sql
optimize silver_shipments zorder by (source_city, shipment_date);

In [0]:
%sql
select * from silver_shipments where source_city = "Pune";

In [0]:
%sql
describe history silver_shipments

#### 2. Speeding up Regional Queries (Partition Pruning)
Scenario: The dashboard team reports that queries filtering for orgin_hub_city with "New York" shipments from the gold_core_curated_tbl table are scanning the entire dataset (Terabytes of data), even though New York is only 5% of the data. This is racking up compute costs.

Task: Re-create the gold_core_curated_tbl table partitioned by orgin_hub_city. Run a query filtering for one city to demonstrate "Partition Pruning" (where Spark skips files that don't match the filter).

Outcome: Verify the partition filtering is applied or not, by performing explain plan, check for the PartitionFilters in the output.

In [0]:
%sql
use prodcatalog_wd36.logistics_wd36

In [0]:
%sql
select * from gold_core_curated_tbl;

In [0]:
%sql
create or replace table gold_core_curated_tbl partitioned by (origin_hub_city) as select * from gold_core_curated_tbl;

In [0]:
%sql
select * from gold_core_curated_tbl where origin_hub_city='London';

In [0]:
%sql
explain select * from gold_core_curated_tbl where origin_hub_city='London';

#### 3. Storage Cost Savings (Vacuum)
Scenario: Your Project pipeline runs every hour, creating many small files and obsolete versions of data. Your storage costs are rising. You need to clean up files that are no longer needed for time travel.

Task: Execute a Vacuum command to remove data files older than the retention threshold.

Outcome: Perform the describe history and find whether vacuum is completed.

In [0]:
%sql
use prodcatalog_wd36.logistics_wd36

In [0]:
%sql
VACUUM gold_core_curated_tbl RETAIN 168 HOURS;

In [0]:
%sql
describe history gold_core_curated_tbl;

####4. Modern Data Layout (Liquid Clustering)
Scenario: You are redesigning the silver_shipments table. You want to avoid the "small files" problem and need a flexible layout that adapts to changing query patterns automatically without rewriting the table.

Task: Re-create the silver_shipments table using Liquid Clustering on the shipment_id column.

Outcome: Liquid Clustering over traditional partitioning when the cardinality of shipment_id is very high.

In [0]:
%sql
CREATE or REPLACE table silver_shipments CLUSTER BY (shipment_id) as select * from silver_shipments;

In [0]:
%sql
describe history silver_shipments;

#### 5. Cost Efficient Environment Cloning (Shallow Clone)
Scenario: The QA team needs to test an update on the gold_core_curated_tbl table. The table is 5TB in size. You cannot afford to duplicate the storage cost just for a test and the update should not affect the copied table.

Task: Create a Shallow Clone of the gold table for the QA team.

Outcome: If we delete records from the source table (gold_core_curated_tbl), will the QA table (gold_core_curated_tbl_qa) be affected? Why or why not?

In [0]:
%sql
CREATE OR REPLACE TABLE gold_core_curated_tbl_qa SHALLOW CLONE gold_core_curated_tbl;

In [0]:
%sql
describe history gold_core_curated_tbl_qa;

In [0]:
%sql
select * from gold_core_curated_tbl_qa;

In [0]:
%sql
drop table gold_core_curated_tbl;

In [0]:
%sql
select * from gold_core_curated_tbl;

In [0]:
%sql
---QA table not affeted because the drops table delete the metadata related to source table not the files.
select * from gold_core_curated_tbl_qa;

In [0]:
%sql
select * from gold_core_curated_tbl limit 2;

In [0]:
%sql
describe history gold_core_curated_tbl;

In [0]:
%sql
----The target QA table inserted new row not showing in this source table
select * from gold_core_curated_tbl where shipment_id='100';

In [0]:
%sql
--The versioning not being updated in the source table after target table inserted with new data
describe history gold_core_curated_tbl

In [0]:
%sql
use prodcatalog_wd36.logistics_wd36;
CREATE or REPLACE table gold_core_curated_test_qa SHALLOW CLONE gold_core_curated_tbl;


In [0]:
%sql
select * from gold_core_curated_test_qa limit 2;

In [0]:
%sql
describe history gold_core_curated_test_qa;

In [0]:
%sql
insert into gold_core_curated_test_qa(shipment_id,role) values ('100','Dataengineer');


In [0]:
%sql
describe history gold_core_curated_test_qa

In [0]:
%sql
select * from gold_core_curated_test_qa where shipment_id='100';


#### 6. Disaster Recovery (Time Travel & Restore)
Scenario: A junior data engineer accidentally ran a logic error that corrupted the gold_core_curated_tbl table 15 minutes ago. You need to revert the table to its previous state immediately.

Task: Use Delta Lake's Restore feature to roll back the table.

Outcome:What is the difference between querying with VERSION AS OF (Time Travel) and running RESTORE?

In [0]:
%sql
describe history gold_core_curated_tbl_qa;

In [0]:
%sql
optimize gold_core_curated_tbl

In [0]:
%sql
describe detail gold_core_curated_tbl

In [0]:
%sql
optimize gold_core_curated_tbl_qa;


In [0]:
%sql 
describe history gold_core_curated_tbl_qa;

In [0]:
%sql
undrop table gold_core_curated_tbl;

In [0]:
%sql
select * from gold_core_curated_tbl;

In [0]:
%sql
describe history gold_core_curated_tbl;

In [0]:
%sql
restore table gold_core_curated_tbl version as of 9;

In [0]:
%sql
describe history gold_core_curated_tbl;

In [0]:
%sql
create or replace table test_tbl (column1 int, column2 string) using delta;

In [0]:
%sql
select * from test_tbl;

In [0]:
%sql
drop table test_tbl;

In [0]:
%sql
select * from test_tbl;

In [0]:
%sql
undrop table test_tbl;

In [0]:
%sql
select * from test_tbl;